In [1]:
import tarfile
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import json
import xml.dom.minidom

from IPython.display import Javascript
from geojson import Feature, LineString, FeatureCollection


%matplotlib notebook

In [2]:
# Load ODV file
filename = 'b2drop/SDN_Elba_SpreadSheet_2.tgz'

In [3]:
# Unpack tar file
tar = tarfile.open(filename, "r:gz")
tar.extractall('tmp/')
TarInfo = tar.getmembers()

In [4]:
# Import file to pandas dataframe
importdata = pd.read_csv(r'tmp/SDN_Elba_SpreadSheet_2.txt', skiprows=71, comment='//<', sep=r'\t', engine='python')

In [5]:
# Concert Dataframe to geojson
data = importdata
cruises = data.Cruise.unique()

data = data.fillna('None')
featcol = []

for cruise in cruises[:5]: 
    cruise_data = data.loc[data['Cruise'] == cruise]
    geo = cruise_data[['Longitude [degrees_east]', 'Latitude [degrees_north]']].values.tolist()
    feat = {
                'type': 'Feature',
                'geometry': {
                    'type': 'LineString', 
                    'coordinates': geo
                },
                'properties': {
                    'name': cruise,
                    'time': cruise_data['yyyy-mm-ddThh:mm:ss.sss'].values.tolist(),
                    'depth': cruise_data['Depth [m]'].values.tolist(),
                    'temp': cruise_data['ITS-90 water temperature [degrees C]'].values.tolist(),
                    'salinity': cruise_data['Water body salinity [per mille]'].values.tolist()
                }
           }
    featcol.append(feat)
FC = FeatureCollection(featcol)


# Create Visualization in Mapbox
Use html to create a div. In Javascript add a mapbox component. When clicking on an element a bokeh plot will appear. 

In [6]:
%%html

<head>
    <meta charset="utf-8" name='viewport' content='initial-scale=1,maximum-scale=1,user-scalable=no' />
    <script src='https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.2/mapbox-gl.js'></script>
    <link href='https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.2/mapbox-gl.css' rel='stylesheet' />
</head>
<div id='map_div' class='layer' style='width: 50vw; height: 50vh'/>
<div id='data' style='width: 50vw; height: 50vh; background-color: white;'/>

<style>
    path { mix-blend-mode: lighten; }
    .leaflet-zoom-animated { isolation: leaflet-zoom-animated; }
</style>

In [7]:
# Make geojson object available in our javscript 
Javascript("""
           window.locs = {};
           """.format(FC))


<IPython.core.display.Javascript object>

In [25]:
from bokeh.embed import components, file_html
from bokeh.models import HoverTool, TapTool, CustomJS, ColumnDataSource
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
import pandas as pd
import numpy as np

def bokeh_plots(data):
#     data = repr(data).replace("'", '').replace('"[', '[').replace(']"', ']')

    data = repr(data).replace("'", '"').replace('"[', '[').replace(']"', ']')
#     data = json.loads(data)  
    data = pd.read_json(data)
    z = data['depth']
    temp = data['temp']
    sal = data['salinity']
    time = pd.to_datetime(data['time'], format='%Y-%m-%dT%H:%M:%S')
    TOOLS = 'pan, wheel_zoom, box_zoom, reset, save, box_select, lasso_select'
    
    left = figure(plot_width=300, plot_height=300, tools=TOOLS, title=r'Temperature (C)', x_axis_type='datetime')
    right = figure(plot_width=300, plot_height=300, y_range=left.y_range, tools=TOOLS, title='Salinity (psu)', x_axis_type='datetime')
    left.circle(temp, time, line_width=2)
    right.circle(sal, time, line_width=2)

#     lL = left.circle(temp, z, line_width=2, hover_alpha=1.0, hover_color='red')
#     lR = right.circle(sal, z, line_width=2, hover_alpha=1.0, hover_color='red')

    p2 = gridplot([[left, right]], title='Data plots', toolbar_location='above',
           plot_width=400, plot_height=300);
    
    script, div = components(p2)
#     show(p2)
    return script, div


# output_notebook()
# bokeh_plots('{"depth":"[0,9.9,19.8,29.8,49.6,74.4,99.2,148.8,198.3,247.89999,297.39999,396.5,495.39999000000006,0,9.9,19.8,29.8,49.6,74.4,99.2,148.8,198.3,247.89999,297.39999,396.5,495.39999000000006,594.40002,0,9.9,19.8,29.8,49.6,74.4,99.2,148.8,198.3,238]","name":"56050711","salinity":"[38.34,38.3,\"None\",38.15,38.03,38.1,38.21,38.3,38.39,38.44,38.48,38.55,38.55,38.3,38.3,38.3,38.26,38.03,38.08,38.18,38.27,38.35,38.43,38.49,38.56,38.55,38.55,38.33,38.31,38.29,38.23,38.04,38.05,38.08,38.17,38.3,38.37]","temp":"[22.8,22.56,22.49,18.49,15.33,13.85,13.51,13.31,13.35,13.36,13.47,13.54,13.49,22.6,22.34,22.27,21.14,15.74,14.16,13.69,13.4,13.32,13.38,13.44,13.58,13.52,13.52,22.5,22.19,21.84,20.79,15.49,14.86,14.15,13.59,13.35,13.33]","time":"[\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:00:00\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T13:59:59\",\"1956-09-23T16:00:00\",\"1956-09-23T16:00:00\",\"1956-09-23T16:00:00\",\"1956-09-23T16:00:00\",\"1956-09-23T16:00:00\",\"1956-09-23T16:00:00\",\"1956-09-23T16:00:00\",\"1956-09-23T16:00:00\",\"1956-09-23T16:00:00\",\"1956-09-23T16:00:00\"]"}')

In [9]:
%%javascript
require.config({
  paths: {
      mapboxgl: 'https://api.tiles.mapbox.com/mapbox-gl-js/v0.44.2/mapbox-gl',
      bootstrap: 'https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min',
      Bokeh: 'http://cdn.pydata.org/bokeh/release/bokeh-0.12.15'
    }
});


<IPython.core.display.Javascript object>

In [36]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;
require(['mapboxgl', 'bootstrap', 'Bokeh'], function(mapboxgl, bootstrap, Bokeh){
    mapboxgl.accessToken = 'pk.eyJ1IjoicnNiYXVtYW5uIiwiYSI6IjdiOWEzZGIyMGNkOGY3NWQ4ZTBhN2Y5ZGU2Mzg2NDY2In0.jycgv7qwF8MMIWt4cT0RaQ';
    var map = new mapboxgl.Map({
        container: 'map_div', // container id
        style: 'mapbox://styles/mapbox/dark-v8', //stylesheet location
        center: [0, 0], // starting position
        zoom: 2 // starting zoom
    });
    console.log(locs)
    map.once('style.load', function(e) {
        map.addLayer({
            'id': 'Cruises',
            'type': 'line',
            'source': {
                'type': 'geojson',
                'data': locs
            },
            'paint': {
                'line-color': 'red',
                'line-width': 2
            }
        })
        map.addLayer({
            'id': 'Cruises-hover',
            'type': 'line',
            'source': {
                'type': 'geojson',
                'data': locs
            },
            'paint': {
                'line-color': 'white',
                'line-width': 4
            },
            'filter': ['==', 'name', '']
        })   
        
        var popup = new mapboxgl.Popup({
            closeButton: false,
            closeOnClick: false
        });
        
        
        
        map.on('mousemove', 'Cruises', (e) => {
            var name = e.features[0].properties.name;
            map.setFilter('Cruises-hover', ['==', 'name', name])
            map.getCanvas().style.cursor = 'pointer'

            popup.setLngLat(e.lngLat)
                .setHTML(name)
                .addTo(map);
        })

        map.on('mouseleave', 'Cruises', (e) => {
            map.setFilter('Cruises-hover', ['==', 'name', ''])
            map.getCanvas().style.cursor = ''
            popup.remove();
        })
        
        map.on('click', 'Cruises', (e) => {
            console.log(JSON.stringify(e.features[0].properties))
            IPython.notebook.kernel.execute('bokeh_plots(' + JSON.stringify(e.features[0].properties) + ')', {
                iopub: {
                    output: function(response) {
                        var output = response.content.data['text/plain'].slice(1, -1).split(',\n');
                        console.log(output)
                        var script = eval(output[0]);
                        var div = eval(output[1]);
                        var data = document.getElementById('data');
                        data.innerHTML = div
                        $('head').append(script);
                    }                
                }
            }, {silent: false, store_history: false, stop_on_error: true}
            );
        })
    });
});

<IPython.core.display.Javascript object>